In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import time 

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

# Настройка Chrome для запуска в безголовом режиме (headless)
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Установка и запуск драйвера
service = ChromeService(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    # Открываем страницу
    url = "https://www.nhl.com/stats/teams?aggregate=0&reportType=game&seasonFrom=20032004&seasonTo=20232024&dateFromSeason&gameType=2&sort=a_gameDate&page=0&pageSize=100"
    driver.get(url)

    # Прокрутка к кнопке "Следующая" и клик по ней
    element = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "button[aria-label='Go to page 2']")
        )
    )
    driver.execute_script("arguments[0].scrollIntoView(true);", element)
    ActionChains(driver).move_to_element(element).click().perform()

    # Ожидание, пока элемент с классом 'rt-table' станет видимым
    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CLASS_NAME, "rt-table"))
    )

    # Получаем HTML-код страницы
    page_source = driver.page_source

    # Сохраняем HTML-код страницы в файл
    with open("nhl_page_dynamic.html", "w", encoding="utf-8") as file:
        file.write(page_source)

finally:
    # Закрываем драйвер
    driver.quit()

In [16]:
from bs4 import BeautifulSoup
import pandas as pd

# Открываем файл с HTML-кодом страницы
with open("nhl_page_dynamic.html", "r", encoding="utf-8") as file:
    html = file.read()

# Парсим HTML с помощью BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

# Найдём таблицу
table = soup.find("table", {"class": "sc-tYrNF joVnOU rt-table"})

# Создадим списки для хранения данных
data = []
headers = []

# Извлекаем заголовки столбцов
header_row = table.find("thead").find_all("tr")[0]
for th in header_row.find_all("th"):
    headers.append(th.get_text())

# Извлекаем строки таблицы
rows = table.find("tbody").find_all("tr")
for row in rows:
    cols = row.find_all("td")
    cols = [col.get_text() for col in cols]
    data.append(cols)

# Создаём DataFrame и сохраняем в CSV
df = pd.DataFrame(data, columns=headers)
df.to_csv("nhl_stats1.csv", index=False)

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

# Настройки Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")  # Запуск браузера в фоновом режиме

# Путь к вашему chromedriver
service = Service("/path/to/chromedriver")  # Замените на путь к вашему chromedriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Открыть начальную страницу
driver.get(
    "https://www.nhl.com/stats/teams?aggregate=0&reportType=game&seasonFrom=20032004&seasonTo=20232024&dateFromSeason&gameType=2&sort=a_gameDate&page=0&pageSize=100"
)

# Дайте странице время для загрузки
time.sleep(5)

# Перейти на следующую страницу
try:
    # Найти кнопку для перехода на следующую страницу и нажать на нее
    next_button = driver.find_element(
        By.CSS_SELECTOR, "button[aria-label='Go to page 2']"
    )
    next_button.click()
except Exception as e:
    print(f"Ошибка при нажатии на кнопку: {e}")

# Подождите некоторое время, чтобы убедиться, что страница загрузилась
time.sleep(5)

# Получите исходный код страницы после перехода
page_source = driver.page_source

# Закройте браузер
driver.quit()

# Теперь можно продолжить обработку HTML-кода страницы

NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
